## Install Dependencies

In [ ]:
seed_value = 42
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

In [ ]:
# pip install wfdb wget tqdm biosppy imbalanced-learn seaborn

## Importing Libraries

In [ ]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import wget
import math
import zipfile
import wfdb as wf
import os
import pickle
import sys
import datetime
import cv2
import random
import itertools
import scipy.io
import random as python_random
import scipy.interpolate as interp
from scipy import signal
from scipy.signal import resample
from scipy.signal import find_peaks
from pathlib import Path
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from scipy.fft import fft, ifft, fftfreq, rfft,irfft, rfftfreq
from sklearn import preprocessing

In [ ]:
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(seed_value)
   tf.random.set_seed(seed_value)
   np.random.seed(seed_value)
   random.seed(seed_value)

In [ ]:
np.random.seed(seed_value)
vec = np.random.randint(1, 10)
print(vec)
random.seed(42)
print(random.random())
tf.random.set_seed(42)
print(tf.random.uniform([1])) 

## Extracting Data

In [ ]:
# all_subject_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22]
test_subject_env_value_str = os.environ.get('TEST_SUBJECT_ARG')
test_subject_env_value = int(test_subject_env_value_str)
test_subject = [test_subject_env_value]
rest_subjects = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22]
rest_subjects.remove(test_subject_env_value)
print(test_subject)
print(rest_subjects)

In [ ]:
SOURCE_PATH = 'DREAMER.mat'

def load_data(subject_id): 
    basl_left = []
    valence_left = []
    arousal_left = []
    
    data_file = scipy.io.loadmat(SOURCE_PATH)
    
    valence_path = (data_file["DREAMER"][0, 0]["Data"][0, subject_id]["ScoreValence"][0, 0])
    arousal_path = (data_file["DREAMER"][0, 0]["Data"][0, subject_id]["ScoreArousal"][0, 0])
    
    print('Loading data for S'+ str(subject_id))
    
    for video in range(0, 18):
        basl_left.append(data_file["DREAMER"][0, 0]["Data"]
                    [0, subject_id]["ECG"][0, 0]
                    ["baseline"][0, 0][video, 0][:, 0])
        stim_left = (data_file["DREAMER"][0, 0]["Data"]
                         [0, subject_id]["ECG"][0, 0]
                         ["stimuli"][0, 0][video, 0][:, 0])

        if (valence_path[video, 0] > arousal_path[video, 0]):
            valence_left.append(stim_left)
        else:
            if (valence_path[video, 0] < arousal_path[video, 0]):
                arousal_left.append(stim_left)
            
    Full_basl_left = list(itertools.chain.from_iterable(basl_left))
    Full_valence_left = list(itertools.chain.from_iterable(valence_left))
    Full_arousal_left = list(itertools.chain.from_iterable(arousal_left))
    
            
    return Full_basl_left, Full_valence_left, Full_arousal_left

##  Data Preprocesssing

In [ ]:
# config
debug = False;

In [ ]:
fs = 256.0
lowcut = 4.0
highcut = 5.0

In [ ]:
def getBand(signal, highcut, lowcut, order, fs):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    s3 = lfilter(b, a, signal)
    return s3
    
def cutBandHelper(signal, highcut, lowcut, order, fs):
    new_signal = signal.copy()
    nyq = 0.5 * fs
    lowcut = lowcut/nyq
    highcut = highcut/nyq
    b, a = butter(order, lowcut, btype = 'low')
    d, c = butter(order, highcut, btype = 'high')
    s1 = lfilter(b, a, signal)
    s2 = lfilter(d, c, new_signal)
    return s1, s2

In [ ]:
def generateData(subject_ids):
    all_subject_data = np.array([]);
    all_subject_labels = np.array([]);
    
    for subject_id in subject_ids:
        if(debug): print("Processing subject id: ", subject_id);
        signal_data = load_data(subject_id)
        
        ecg_baseline = signal_data[0]
        ecg_valence = signal_data[1]
        ecg_arousal = signal_data[2]
        
        
        if(debug):
            print("Raw ecg_baseline:", ecg_baseline)
            print("Raw ecg_valence:", ecg_valence)
            print("Raw ecg_arousal:", ecg_arousal)
            
        ecg_baseline_data = np.array(ecg_baseline)
        ecg_valence_data = np.array(ecg_valence)
        ecg_arousal_data = np.array(ecg_arousal)
        
        ecg_arousal_data = ecg_arousal_data[0:208128]

        if(debug):
            plt.plot(ecg_baseline_data)
            plt.show()

            plt.plot(ecg_arousal_data)
            plt.show()
            
    return ecg_baseline_data, ecg_arousal_data

## Test Data

In [ ]:
test_all_subject_data = np.array([]);
test_ecg_baseline_data, test_ecg_arousal_data = generateData(test_subject)

test_baseline_filtered = getBand(test_ecg_baseline_data, highcut, lowcut, 1, fs)
test_arousal_filtered = getBand(test_ecg_arousal_data, highcut, lowcut, 1, fs)

window_size = 256
window_shift = 256

test_heart_beat_base = []
for i in range(0,len(test_ecg_baseline_data) - window_size,window_shift):
    test_heart_beat_base.append(test_ecg_baseline_data[i:window_size + i])
test_heart_beat_base.pop()

test_heart_beat_arousal = []
for i in range(0,len(test_ecg_arousal_data) - window_size,window_shift):
    test_heart_beat_arousal.append(test_ecg_arousal_data[i:window_size + i])
test_heart_beat_arousal.pop()


for idx, idxval in enumerate(test_heart_beat_base):
    test_heart_beat_base[idx] = (test_heart_beat_base[idx] - test_heart_beat_base[idx].min()) / test_heart_beat_base[idx].ptp() # Normalize the readings to a 0-1 range 
    test_heart_beat_base[idx] = np.append(test_heart_beat_base[idx], 0.0) #baseline = 0

for idx, idxval in enumerate(test_heart_beat_arousal):
    test_heart_beat_arousal[idx] = (test_heart_beat_arousal[idx] - test_heart_beat_arousal[idx].min()) / test_heart_beat_arousal[idx].ptp() # Normalize the readings to a 0-1 range 
    test_heart_beat_arousal[idx] = np.append(test_heart_beat_arousal[idx], 1.0) #arousal = 1 

test_heart_beat_all = np.concatenate((test_heart_beat_base, test_heart_beat_arousal), axis=0)
test_subject_data = np.array(list(test_heart_beat_all[:]), dtype=float)

if(test_all_subject_data.size == 0):
    test_all_subject_data = test_subject_data
else:
    if(test_subject_data.size != 0):
        test_all_subject_data = np.concatenate((test_all_subject_data, test_subject_data), axis=0)

print("New data shape", test_subject_data.shape, "Total Shape: ", test_all_subject_data.shape)

#SMOTE to balance the data
test_df_final_data_X = pd.DataFrame(data=test_all_subject_data[:, :-1])
test_df_final_data_Y = pd.DataFrame(data=test_all_subject_data[:,-1])

test_smote = SMOTE(sampling_strategy='not majority')
test_data, test_labels = test_smote.fit_resample(test_df_final_data_X, test_df_final_data_Y)

In [ ]:
test_labels.value_counts()

In [ ]:
print("Test data shape:", test_data.shape)
print("Test labels shape:", test_labels.shape)
print("Dimension:", test_labels.ndim)

In [ ]:
test_labels = pd.DataFrame(test_labels).to_numpy()
y_test = test_labels.reshape(-1)

In [ ]:
print("Test labels shape:", y_test.shape)
print("Dimension:", y_test.ndim)

In [ ]:
#Reshape test data to (n_samples, 256, 1), where each sample is of size (256, 1)
X_test = np.array(test_data).reshape(test_data.shape[0], test_data.shape[1], 1)

In [ ]:
print('size of X_test:', X_test.shape)
print('size of y_test:', y_test.shape)

print('Test:', Counter(y_test))

## Training Data

In [ ]:
all_subject_data = np.array([]);

for idx, idxval in enumerate(rest_subjects):
    ecg_baseline_data, ecg_arousal_data = generateData([idxval])
    
    baseline_s1, baseline_s2 = cutBandHelper(ecg_baseline_data, highcut, lowcut, 1, fs)
    arousal_s1, arousal_s2 = cutBandHelper(ecg_arousal_data, highcut, lowcut, 1, fs)
    
    new_baseline = np.sum([baseline_s1, test_baseline_filtered, baseline_s2], axis=0)
    new_arousal = np.sum([arousal_s1, test_arousal_filtered, arousal_s2], axis=0)
    
    print('reformed base size:', new_baseline.size)
    print('reformed valence size:', new_arousal.size)
    
    window_size = 256
    window_shift = 256

    heart_beat_base = []
    for i in range(0,len(new_baseline) - window_size,window_shift):
        heart_beat_base.append(new_baseline[i:window_size + i])
    heart_beat_base.pop()

    heart_beat_arousal = []
    for i in range(0,len(new_arousal) - window_size,window_shift):
        heart_beat_arousal.append(new_arousal[i:window_size + i])
    heart_beat_arousal.pop()


    for idx, idxval in enumerate(heart_beat_base):
        heart_beat_base[idx] = (heart_beat_base[idx] - heart_beat_base[idx].min()) / heart_beat_base[idx].ptp() # Normalize the readings to a 0-1 range 
        heart_beat_base[idx] = np.append(heart_beat_base[idx], 0.0) #baseline = 0

    for idx, idxval in enumerate(heart_beat_arousal):
        heart_beat_arousal[idx] = (heart_beat_arousal[idx] - heart_beat_arousal[idx].min()) / heart_beat_arousal[idx].ptp() # Normalize the readings to a 0-1 range 
        heart_beat_arousal[idx] = np.append(heart_beat_arousal[idx], 1.0) #arousal = 1 

    heart_beat_all = np.concatenate((heart_beat_base, heart_beat_arousal), axis=0)
    subject_data = np.array(list(heart_beat_all[:]), dtype=float)

    if(all_subject_data.size == 0):
        all_subject_data = subject_data
    else:
        if(subject_data.size != 0):
            all_subject_data = np.concatenate((all_subject_data, subject_data), axis=0)

    print("New data shape", subject_data.shape, "Total Shape: ", all_subject_data.shape)

#SMOTE to balance the data
df_final_data_X = pd.DataFrame(data=all_subject_data[:, :-1])
df_final_data_Y = pd.DataFrame(data=all_subject_data[:,-1])

smote = SMOTE(sampling_strategy='not majority')
train_data, train_labels = smote.fit_resample(df_final_data_X, df_final_data_Y)

In [ ]:
train_labels.value_counts()

In [ ]:
print("Train data shape:", train_data.shape)
print("Train labels shape:", train_labels.shape)
print("Dimension:", train_labels.ndim)

In [ ]:
train_labels = pd.DataFrame(train_labels).to_numpy()
train_labels = train_labels.reshape(-1)

In [ ]:
print("Train labels shape:", train_labels.shape)
print("Dimension:", train_labels.ndim)

## Train and Validation data split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.3, random_state=42)

In [ ]:
#Reshape train and validation data to (n_samples, 256, 1), where each sample is of size (256, 1)
X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = np.array(X_val).reshape(X_val.shape[0], X_val.shape[1], 1)

In [ ]:
print('size of X_train:', X_train.shape)
print('size of X_val:', X_val.shape)

print('size of y_train:', y_train.shape)
print('size of y_val:', y_val.shape)

print('Train:', Counter(y_train), '\nVal:', Counter(y_val))

## 1D CNN Model Architecture

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, BatchNormalization, MaxPool1D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
reset_random_seeds()
# kernel_regularizer =tf.keras.regularizers.l2(l=0.004)
# initializer = tf.keras.initializers.HeUniform()
cnn_model = tf.keras.models.Sequential()
cnn_model.add(Conv1D(filters=5, kernel_size=(5,), padding='same',  activation='relu', input_shape= (X_train.shape[1],1)))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPool1D(pool_size=(2,), strides=2))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv1D(filters=10, kernel_size=(5,), padding='same', activation='relu', kernel_regularizer =tf.keras.regularizers.l2(l=0.004)))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPool1D(pool_size=(2,), strides=2))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv1D(filters=15, kernel_size=(5,), padding='same', activation='relu', kernel_regularizer =tf.keras.regularizers.l2(l=0.004)))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPool1D(pool_size=(2,), strides=2))
cnn_model.add(BatchNormalization())
cnn_model.add(Flatten())
cnn_model.add(Dense(units = 128, activation='relu', kernel_regularizer =tf.keras.regularizers.l2(l=0.004)))
cnn_model.add(Dense(units = 1, activation='sigmoid'))
cnn_model.summary()

In [ ]:
print(cnn_model.layers[0])
print(cnn_model.layers[0].get_weights()[0])
print(cnn_model.layers[0].get_weights()[0].shape)
print(cnn_model.layers[0].get_weights()[1])

In [ ]:
# weights = 0.001 * (np.random.rand(5, 1, 5))
# print(weights)
# bias = np.random.rand(5)
# print(bias)

In [ ]:
# cnn_model.layers[0].set_weights([weights, bias])

In [ ]:
epochs=50
batch_size = 32

In [ ]:
def step_decay(epoch):
  initial_lrate = 0.005
  drop = 0.6
  epochs_drop = 10.0
  lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
  return lrate

lrate = LearningRateScheduler(step_decay)
callbacks_list = [lrate]

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.0, momentum=0.6, nesterov=False)

cnn_model.compile(optimizer= opt, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = cnn_model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (X_val, y_val), callbacks=callbacks_list, verbose=2)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('1D CNN Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

max_epoch = len(history.history['accuracy'])+1
epoch_list = list(range(1,max_epoch))
ax1.plot(epoch_list, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epoch_list, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(np.arange(1, max_epoch, 5))
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epoch_list, history.history['loss'], label='Train Loss')
ax2.plot(epoch_list, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(np.arange(1, max_epoch, 5))
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
cnn_model.evaluate(X_test, y_test)

## Classification Report

In [ ]:
print(y_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_test_preds = np.around(cnn_model.predict(X_test))
# print(y_test_preds)

In [ ]:
print(y_test_preds)

In [ ]:
y_test_preds.shape

In [ ]:
import sklearn.metrics as metrics

In [ ]:
print(metrics.classification_report(y_test, y_test_preds))

In [ ]:
# reset_random_seeds()
# # kernel_regularizer =tf.keras.regularizers.l2(l=0.004)
# # initializer = tf.keras.initializers.HeUniform()
# cnn_model = tf.keras.models.Sequential()
# cnn_model.add(Conv1D(filters=5, kernel_size=(5,), padding='same',  activation='relu', input_shape= (X_train.shape[1],1)))
# cnn_model.add(BatchNormalization())
# cnn_model.add(MaxPool1D(pool_size=(2,), strides=2))
# cnn_model.add(BatchNormalization())
# cnn_model.add(Conv1D(filters=10, kernel_size=(5,), padding='same', activation='relu', kernel_regularizer =tf.keras.regularizers.l2(l=0.004)))
# cnn_model.add(BatchNormalization())
# cnn_model.add(MaxPool1D(pool_size=(2,), strides=2))
# cnn_model.add(BatchNormalization())
# cnn_model.add(Conv1D(filters=15, kernel_size=(5,), padding='same', activation='relu', kernel_regularizer =tf.keras.regularizers.l2(l=0.004)))
# cnn_model.add(BatchNormalization())
# cnn_model.add(MaxPool1D(pool_size=(2,), strides=2))
# cnn_model.add(BatchNormalization())
# cnn_model.add(Conv1D(filters=20, kernel_size=(5,), padding='same', activation='relu', kernel_regularizer =tf.keras.regularizers.l2(l=0.004)))
# cnn_model.add(BatchNormalization())
# cnn_model.add(MaxPool1D(pool_size=(2,), strides=2))
# cnn_model.add(BatchNormalization())
# cnn_model.add(Flatten())
# cnn_model.add(Dense(units = 128, activation='relu', kernel_regularizer =tf.keras.regularizers.l2(l=0.004)))
# cnn_model.add(Dense(units = 2, activation='softmax'))
# cnn_model.summary()